In [10]:
from ortools.linear_solver import pywraplp
import openpyxl
import os
import sys
import time
import datetime
from datetime import datetime as dt

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import pandas as pd

from random import seed
from random import random

In [11]:
path = './advanced/data/'

In [12]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

DEM = pd.read_excel(path+'dataLarge.xlsx',sheet_name='DEM')
CAPA = pd.read_excel(path+'dataLarge.xlsx',sheet_name='CAPA')
PCOST = pd.read_excel(path+'dataLarge.xlsx',sheet_name='PCOST')
SCOST = pd.read_excel(path+'dataLarge.xlsx',sheet_name='SCOST')
ISTOCK = pd.read_excel(path+'dataLarge.xlsx',sheet_name='ISTOCK')
ITEM = DEM['ITEM'].drop_duplicates().to_list()
RES = CAPA['RES'].drop_duplicates().to_list()
TIME = DEM['TIME'].drop_duplicates().to_list()

endDateTime = dt.now()
print("End : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print("Load Data :" + str(endDateTime-startDateTime))

Start : 2023-Jan-31 17:12:45.606261
End : 2023-Jan-31 17:12:46.179745
Load Data :0:00:00.573484


In [14]:
arr_DEM = DEM.to_numpy()

In [15]:
arr_DEM

array([[  1,   1, 300],
       [  1,   2, 400],
       [  1,   3, 500],
       ...,
       [100,  48, 400],
       [100,  49, 300],
       [100,  50, 200]])